In [62]:
import pandas as pd
import torch
import torchvision
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
from torch.utils.data import Dataset,DataLoader

In [63]:
df=pd.read_csv('/content/titanic.csv')

In [64]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [65]:
df['Sex']=LabelEncoder().fit_transform(df['Sex'])
df['Embarked']=LabelEncoder().fit_transform(df['Embarked'])

In [66]:
df.isna().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [67]:
df['Age'].fillna(df['Age'].mean(), inplace=True)

<ipython-input-67-7ee7fe972bc2>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].mean(), inplace=True)


In [68]:
df.drop(['Cabin'],axis=1,inplace=True)

In [69]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,2
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,2


In [70]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X=df[features].values
y=df['Survived'].values

In [71]:
scaler=StandardScaler()
X=scaler.fit_transform(X)

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [73]:
class TitanicDataset(Dataset):
  def __init__(self,features,labels):
    self.features=torch.tensor(features,dtype=torch.float32)
    self.labels=torch.tensor(labels,dtype=torch.long)

  def __len__(self):
    return len(self.labels)

  def __getitem__(self,index):
    return self.features[index] , self.labels[index]

train_dataset=TitanicDataset(X_train,y_train)
test_dataset=TitanicDataset(X_test,y_test)

In [74]:
trainloader=DataLoader(train_dataset,batch_size=32,shuffle=True)
testloader=DataLoader(test_dataset,batch_size=32,shuffle=False)

In [75]:
class NeuralNetTitanic(nn.Module):
  def __init__(self,input_size,hidden_size,output_size):
    super(NeuralNetTitanic,self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, output_size)

  def forward(self, x):
      x = self.fc1(x)
      x = self.relu(x)
      x = self.fc2(x)
      return x


In [76]:
input_size=len(features)
hidden_size=64
output_size=2
model=NeuralNetTitanic(input_size,hidden_size,output_size)

In [77]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
optim=torch.optim.SGD(model.parameters(),lr=0.001)

In [78]:
num_epochs=20
for epoch in range(num_epochs):
  for i ,(inputs,labels) in enumerate(trainloader):
    outputs=model(inputs)
    loss=criterion(outputs,labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/20], Loss: 0.6012
Epoch [2/20], Loss: 0.5073
Epoch [3/20], Loss: 0.4041
Epoch [4/20], Loss: 0.2794
Epoch [5/20], Loss: 0.2058
Epoch [6/20], Loss: 0.7279
Epoch [7/20], Loss: 0.5469
Epoch [8/20], Loss: 0.2720
Epoch [9/20], Loss: 0.5927
Epoch [10/20], Loss: 0.5271
Epoch [11/20], Loss: 0.1598
Epoch [12/20], Loss: 0.3960
Epoch [13/20], Loss: 0.2675
Epoch [14/20], Loss: 0.4954
Epoch [15/20], Loss: 0.2132
Epoch [16/20], Loss: 0.1956
Epoch [17/20], Loss: 0.2543
Epoch [18/20], Loss: 0.3461
Epoch [19/20], Loss: 0.4502
Epoch [20/20], Loss: 0.2564


In [79]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for features, labels in testloader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total:.2f}%')


Accuracy: 80.45%


In [85]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Put the model in evaluation mode
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for features, labels in testloader:
        # Forward pass
        outputs = model(features)
        _, predicted = torch.max(outputs, 1)

        # Collect predictions and true labels
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy, precision, recall, and F1-score
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

# Optionally, display the confusion matrix and classification report
conf_matrix = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", conf_matrix)

class_report = classification_report(all_labels, all_preds, target_names=['Not Survived', 'Survived'])
print("Classification Report:\n", class_report)


Accuracy: 80.45%
Precision: 0.80
Recall: 0.70
F1 Score: 0.75
Confusion Matrix:
 [[92 13]
 [22 52]]
Classification Report:
               precision    recall  f1-score   support

Not Survived       0.81      0.88      0.84       105
    Survived       0.80      0.70      0.75        74

    accuracy                           0.80       179
   macro avg       0.80      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179

